In [1]:
# Importing libraries

import numpy as np
import pandas as pd
from datetime import datetime, date
import re

from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# Scrap players data (all history)

players_name = ['113', 'Elyoya']

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
data_table = []

for player_name in players_name:
    url = f"https://oracleselixir.com/player/{player_name}/statsBySplit"
    driver.get(url)

    # Find the table on the page (adjust the table selection based on the specific website structure)
    div_element = driver.find_elements(By.CSS_SELECTOR, "div.sc-eFRcpv.gDBTwW")

    player_original_name = driver.find_element(By.CLASS_NAME, "sc-dovdUy.gBqzXj").text

    for item in div_element:
        data_list = item.text.split("\n")
        data_table.append([player_name] + [player_original_name] + data_list)

# Close the browser
driver.quit()

In [5]:
columns_name = ["Player", "Player original name", "Event", "Team", "Position", "Games Played", "Win percentage", "KDA", "KP", "DTH%", "GD@10", "XPD@10", "CSPM", "DPM", "DMG%", "GOLD%", "WPM", "WCPM"]
df_players = pd.DataFrame(data_table, columns = columns_name)

In [9]:
def extract_year(text):
    year_pattern = r'\b(19|20)\d{2}\b'  # Matches years in the range 1900-2099
    matches = re.search(year_pattern, text)
    if matches:
        return matches.group(0)
    else:
        return None

df_players['Year'] = df_players['Event'].apply(extract_year)

In [16]:
df_players.head()

,Player,Player original name,Event,Team,Position,Games Played,Win percentage,KDA,KP,DTH%,GD@10,XPD@10,CSPM,DPM,DMG%,GOLD%,WPM,WCPM,Year
0,113,Doğukan Balcı,LEC 2023 Summer Season,Astralis,Jungle,9,33%,2.7,76.2%,18.6%,21,18,5.0,302,16.2%,19.0%,0.28,0.23,2023
1,113,Doğukan Balcı,LEC 2023 Spring Groups,Astralis,Jungle,7,29%,1.6,82.7%,25.9%,-463,-317,3.9,337,15.1%,14.3%,0.24,0.21,2023
2,113,Doğukan Balcı,LEC 2023 Spring Season,Astralis,Jungle,9,67%,4.0,75.6%,20.8%,-245,-318,4.4,339,16.2%,17.4%,0.50,0.28,2023
3,113,Doğukan Balcı,LEC 2023 Winter Groups,Astralis,Jungle,8,38%,2.5,67.6%,24.8%,-179,-142,4.4,280,13.0%,15.8%,0.22,0.45,2023
4,113,Doğukan Balcı,LEC 2023 Winter Season,Astralis,Jungle,9,33%,2.5,72.2%,20.1%,-111,-161,4.8,263,13.1%,17.1%,0.27,0.32,2023


In [21]:
# Save as csv
df_players.to_csv('C:/Users/adars/OneDrive/Escritorio/ProjecteLolShiny/Data/PlayerStats.csv', index = False)